# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846906810772                   -0.70    4.5         
  2   -7.852323403294       -2.27       -1.53    1.0   25.0ms
  3   -7.852614496990       -3.54       -2.57    1.5   27.9ms
  4   -7.852646033210       -4.50       -2.89    2.5   35.9ms
  5   -7.852646494741       -6.34       -3.15    1.0   26.3ms
  6   -7.852646678279       -6.74       -4.00    1.0   26.2ms
  7   -7.852646686306       -8.10       -5.21    1.8   31.2ms
  8   -7.852646686722       -9.38       -5.37    2.5   35.9ms
  9   -7.852646686729      -11.10       -6.12    1.2   28.6ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846844953947                   -0.70           4.5         
  2   -7.852538690683       -2.24       -1.64   0.80    2.2    288ms
  3   -7.852635486537       -4.01       -2.76   0.80    1.0   24.5ms
  4   -7.852646525757       -4.96       -3.26   0.80    2.0   31.1ms
  5   -7.852646674339       -6.83       -4.09   0.80    1.5    111ms
  6   -7.852646686376       -7.92       -4.69   0.80    2.0   30.1ms
  7   -7.852646686718       -9.47       -5.51   0.80    1.8   30.2ms
  8   -7.852646686730      -10.94       -6.56   0.80    2.0   31.8ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.402532e+01     3.436404e+00
 * time: 0.06357383728027344
     1     1.210786e+00     2.042389e+00
 * time: 0.30402398109436035
     2    -1.373130e+00     2.495529e+00
 * time: 0.32953381538391113
     3    -3.719376e+00     2.008643e+00
 * time: 0.3656017780303955
     4    -4.988254e+00     1.620923e+00
 * time: 0.4019958972930908
     5    -6.607869e+00     1.107377e+00
 * time: 0.43811798095703125
     6    -7.274695e+00     4.095604e-01
 * time: 0.47429895401000977
     7    -7.593414e+00     1.726975e-01
 * time: 0.49985790252685547
     8    -7.745537e+00     1.632310e-01
 * time: 0.5249300003051758
     9    -7.808661e+00     7.446592e-02
 * time: 0.5504608154296875
    10    -7.833580e+00     5.176060e-02
 * time: 0.5757138729095459
    11    -7.844207e+00     4.133822e-02
 * time: 0.6010069847106934
    12    -7.849063e+00     2.653339e-02
 * time: 0.6267538070678711
    13    -7.851433e+00     1.715927e-02
 * time: 0.6522

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846931763061                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645937968                   -1.64         
  2   -7.852646686730       -6.13       -3.72    2.10s
  3   -7.852646686730      -13.35       -7.26    160ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 5.228281510472884e-7
|ρ_newton - ρ_scfv| = 1.704790605815461e-7
|ρ_newton - ρ_dm|   = 1.590016362455079e-9
